In [1]:
import time
!pip install ccxt
import ccxt
from time import gmtime, strftime
import datetime
import pandas as pd
print(ccxt.exchanges)

     |████████████████████████████████| 2.1MB 12.1MB/s 
     |████████████████████████████████| 1.3MB 46.5MB/s 
     |████████████████████████████████| 163kB 48.7MB/s 
     |████████████████████████████████| 3.2MB 44.3MB/s 
     |████████████████████████████████| 143kB 49.1MB/s 
     |████████████████████████████████| 235kB 41.3MB/s 
  Created wheel for yarl: filename=yarl-1.1.0-cp37-cp37m-linux_x86_64.whl size=241454 sha256=edd937b00fdd2fa8289014085abd9600abee696c1423ecf0a04ef6421de61322
  Stored in directory: /root/.cache/pip/wheels/96/26/b9/7972042a1a69556ba7f38b8a301dc3a9ddac1b8f61f02fbfd8
Successfully built yarl
['aax', 'aofex', 'ascendex', 'bequant', 'bibox', 'bigone', 'binance', 'binanceus', 'bit2c', 'bitbank', 'bitbay', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitkk', 'bitmart', 'bitmex', 'bitpanda', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bitz', 'bl3p', 'bleutrade', 'braziliex', 'btcalpha', 'btcbox', 'btcmarkets', 'b

In [60]:
# We define a function to get the bar from one currency to another. 
# We get at most 3000 data each time due to the exchange's transmision length limitation.
def get_bar_from_to(ex,symbol,period,start_time='2017-08-17 00:00:00',end_time='2021-03-05 23:59:59',length=30000,maxLoop=20):
  data = []
  stime = ex.parse8601(start_time) # ISO8601 is the format of time
  etime = ex.parse8601(end_time)
  loop_i = 0
  if ex.has['fetchOHLCV']:
    while stime < etime and loop_i < maxLoop:
      loop_i += 1
      try:
        ohlcvs = ex.fetch_ohlcv(symbol, period, stime, limit=length)
        #print(ex.iso8601(ex.milliseconds()),'loop_i=',loop_i, ' ,Fetched', len(ohlcvs), 'candles')
        if len(ohlcvs) > 1:
          first = ohlcvs[0][0]
          last = ohlcvs[-1][0]
          #print('[',ex.iso8601(first),'--', ex.iso8601(last),']')
          stime = int(last+ (last - first)/(len(ohlcvs)-1))
          data += ohlcvs
          time.sleep(1)
        else:
          return data
      except Exception as e:
        print(str(e))
  return data

from datetime import datetime

# format data as dataframe of pandas
# USDT: started from 2015-02-25
# BTC: started from 2014-09-17
# ETH: started from 2015-08-07
# the start time should not be prior to: 2015-08-07
# in ccxt, the start time seems not be prior to 2017-08-17
# 'BTC/USDT': the earliest is: 2017-08-17
# 'ETH/USDT': the earliest is: 2017-08-17
# start_time = '2017-08-17 00:00:00'
# end_time = '2021-03-05 23:59:59'
def getData(exchange, symbol, period, start_time, end_time, cols=['time','open','high','low','close','volume']):
  # set the start time
  # 就2015-08-07到2021-03-31 -------------- project
  """start_time = '2015-08-17 00:00:00'
  if period == '1d':
    start_time = '2017-08-17 00:00:00'
  elif period == '1h':
    start_time = '2020-01-01 00:00:00'
  # set the end time
  end_time = '2021-03-31 23:59:59'"""

  data = get_bar_from_to(exchange, symbol, period, start_time, end_time)
  test = pd.DataFrame(data, columns=cols)

  # get the data before the end_time
  etime = exchange.parse8601(end_time)
  test = test.loc[ test['time'] <= etime ]
  # convert the time format
    # method 1
    # test['date'] = (test['time']/1000).apply(datetime.fromtimestamp)
    # method 2
  test['day'] = test['time'].apply(lambda x: exchange.iso8601(x)[:10])
  if period == '1h' or period == '1m':
    test['hour'] = test['time'].apply(lambda x: exchange.iso8601(x)[:10] + ' ' + exchange.iso8601(x)[11:16])
  return test

# for a dataframe, get its columns (features)
def getCols(df):
  cols = []
  for col in df:
    cols.append(col)
  return cols

# get the copied data
def getCopy(df):
  cols = getCols(df)
  ti = pd.DataFrame()
  if 'hour' in cols: # hourly data
    ti = df[ ['day', 'hour', 'close'] ].copy()
  else: # daily data
    ti = df[ ['day', 'close'] ].copy()
  return ti

In [15]:
# we set the exchange to binance
exchange = ccxt.binance()

# daily data
btc_day = getData(exchange, 'BTC/USDT', '1d', '2015-08-17 00:00:00', '2021-03-31 23:59:59')
eth_day = getData(exchange, 'ETH/USDT', '1d', '2015-08-17 00:00:00', '2021-03-31 23:59:59')

# daily data shape:  BTC/USDT (1297, 7), ETH/USDT (1297, 7)
print( "data shape: ", btc_day.shape, eth_day.shape)


data shape:  (1323, 7) (1323, 7)
data shape:  (10919, 8) (10919, 8)


In [46]:
# hourly data
st = '2017-08-17 00:00:00'
#et = '2021-03-31 23:59:59'
et = '2018-03-31 23:59:59'
btc_hour = getData(exchange, 'BTC/USDT', '1h', st, et)
eth_hour = getData(exchange, 'ETH/USDT', '1h', st, et)

print( "data shape: ", btc_hour.shape, eth_hour.shape)

data shape:  (5401, 8) (5401, 8)


In [47]:
btc_hour.head()

,time,open,high,low,close,volume,day,hour
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17,2017-08-17 04:00
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17,2017-08-17 05:00
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17,2017-08-17 06:00
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17,2017-08-17 07:00
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17,2017-08-17 08:00


In [38]:
# hourly data
st = '2019-12-02 20:00:00'
#et = '2021-03-31 23:59:59'
et = '2021-03-31 23:59:59'
btc_hour1 = getData(exchange, 'BTC/USDT', '1h', st, et)
eth_hour1 = getData(exchange, 'ETH/USDT', '1h', st, et)

print( "data shape: ", btc_hour1.shape, eth_hour1.shape)

data shape:  (11618, 8) (11618, 8)


In [44]:
btc_hour.to_csv('btc_hour.csv')
eth_hour.to_csv('eth_hour.csv')

In [50]:
btc_hour1.index += 19999
eth_hour1.index += 19999

In [52]:
btc_hour1.index += 1
eth_hour1.index += 1

In [56]:
btc_hour1.tail()
eth_hour1.tail()

,time,open,high,low,close,volume,day,hour
31613,1617217200000,1891.90,1917.77,1891.73,1900.63,68418.66609,2021-03-31,2021-03-31 19:00
31614,1617220800000,1900.63,1940.00,1891.92,1939.07,64382.45547,2021-03-31,2021-03-31 20:00
31615,1617224400000,1939.11,1947.29,1923.86,1929.99,35865.19137,2021-03-31,2021-03-31 21:00
31616,1617228000000,1930.00,1936.53,1903.34,1903.97,35335.71451,2021-03-31,2021-03-31 22:00
31617,1617231600000,1903.97,1924.21,1901.62,1919.37,21224.77764,2021-03-31,2021-03-31 23:00


In [57]:
btc_hour1.to_csv('btc_hour.csv', mode='a', header=False)
eth_hour1.to_csv('eth_hour.csv', mode='a', header=False)

In [13]:
btc_day.to_csv('btc_day.csv')
eth_day.to_csv('eth_day.csv')

In [61]:
# minutely
st = '2017-08-17 00:00:00'
#et = '2021-03-31 23:59:59'
et = '2018-03-31 23:59:59'

btc_hour1 = getData(exchange, 'BTC/USDT', '1m', st, et)
eth_hour1 = getData(exchange, 'ETH/USDT', '1m', st, et)



print( "data shape: ", btc_hour1.shape, eth_hour1.shape)

data shape:  (20000, 8) (20000, 8)


In [63]:
btc_hour1.tail()

,time,open,high,low,close,volume,day,hour
19995,1504142100000,4570.65,4570.65,4570.65,4570.65,0.0,2017-08-31,2017-08-31 01:15
19996,1504142160000,4570.65,4570.65,4570.65,4570.65,0.0,2017-08-31,2017-08-31 01:16
19997,1504142220000,4570.65,4570.65,4570.65,4570.65,0.0,2017-08-31,2017-08-31 01:17
19998,1504142280000,4570.65,4570.65,4570.65,4570.65,0.0,2017-08-31,2017-08-31 01:18
19999,1504142340000,4570.65,4570.65,4570.65,4570.65,0.0,2017-08-31,2017-08-31 01:19
